In [1]:
from datetime import date

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
sns.set(style='white', context='notebook', palette='deep')  
%matplotlib inline

In [59]:
#1754884 record,1053282 with coupon_id,9738 coupon. date_received:20160101~20160615,date:20160101~20160630, 539438 users, 8415 merchants
off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',header=None)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
#2050 coupon_id. date_received:20160701~20160731, 76309 users(76307 in trainset, 35965 in online_trainset), 1559 merchants(1558 in trainset)
off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=None)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

off_train.drop(index=0,inplace=True)
off_test.drop(index=0,inplace=True)

/Users/jimmy233/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [60]:
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
1,1439408,2632,NaN,NaN,0,NaN,20160217
2,1439408,4663,11002,150:20,1,20160528,NaN
3,1439408,2632,8591,20:1,0,20160217,NaN
4,1439408,2632,1078,20:1,0,20160319,NaN
5,1439408,2632,8591,20:1,0,20160613,NaN


# Data Split

In [61]:
off_train.fillna('null',inplace=True)

def tran_int_str(col):
    if col=='null':
        return col
    else:
        return str(int(float(col)))

off_train['date'] = off_train['date'].apply(tran_int_str)
off_train['date_received'] = off_train['date_received'].apply(tran_int_str)

In [62]:
dataset3 = off_test
feature3 = off_train[(
    (off_train.date >= '20160315') & (off_train.date <= '20160630')) | (
        (off_train.date == 'null') & (off_train.date_received >= '20160315') &
        (off_train.date_received <= '20160630'))]
dataset2 = off_train[(off_train.date_received >= '20160515')
                     & (off_train.date_received <= '20160615')]
feature2 = off_train[
    (off_train.date >= '20160201') & (off_train.date <= '20160514')
    | ((off_train.date == 'null') & (off_train.date_received >= '20160201') &
       (off_train.date_received <= '20160514'))]
dataset1 = off_train[(off_train.date_received >= '20160414')
                     & (off_train.date_received <= '20160514')]
feature1 = off_train[
    (off_train.date >= '20160101') & (off_train.date <= '20160413')
    | ((off_train.date == 'null') & (off_train.date_received >= '20160101') &
       (off_train.date_received <= '20160413'))]

# Extract Feature(1.Merchant Related and 5. Other Feature)

## dataset 3 other feature

In [63]:
t = dataset3[['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
# t 此月用户领取全部优惠券数目

<ipython-input-63-a2acdd02fff2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['this_month_user_receive_all_coupon_count'] = 1


In [65]:
t1 = dataset3[['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
# t1 此月用户领取相同优惠券数目

<ipython-input-65-f37557c4fba2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['this_month_user_receive_same_coupon_count'] = 1


In [68]:
t2 = dataset3[['user_id', 'coupon_id', 'date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(
    ['user_id',
     'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s: len(s.split(':')))
t2 = t2[t2.receive_number > 1]        
t2['max_date_received'] = t2.date_received.apply(
    lambda s: max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(
    lambda s: min([int(d) for d in s.split(':')]))
t2 = t2[['user_id', 'coupon_id', 'max_date_received', 'min_date_received']]
#t2 用户同一优惠券领取最大最小时间(leakage)

/Users/jimmy233/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [70]:
t3 = dataset3[['user_id', 'coupon_id', 'date_received']]
t3 = pd.merge(t3, t2, on=['user_id', 'coupon_id'], how='left')
t3.date_received = t3.date_received.apply(lambda x : float(x))
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received


def is_firstlastone(x):
    if x == 0:
        return 1
    elif x > 0:
        return 0
    else:
        return -1  #those only receive once


t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(
    is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(
    is_firstlastone)
t3 = t3[[
    'user_id', 'coupon_id', 'date_received',
    'this_month_user_receive_same_coupon_lastone',
    'this_month_user_receive_same_coupon_firstone'
]]

t3['date_received'] = t3['date_received'].apply(lambda x : str(int(x)))   #将t3的时间格式转换为str
#t3 用户此优惠券为本月第一次领取或者最后一次领取

In [73]:
t4 = dataset3[['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
# t4 当天接受优惠券总数

<ipython-input-73-c1df799ee268>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t4['this_day_user_receive_all_coupon_count'] = 1


In [75]:
t5 = dataset3[['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
# t5 当天接受相同优惠券数目

<ipython-input-75-a76cfc230875>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t5['this_day_user_receive_same_coupon_count'] = 1


In [76]:
t6 = dataset3[['user_id', 'coupon_id', 'date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(
    ['user_id',
     'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
t6.rename(columns={'date_received': 'dates'}, inplace=True)
# t6 同一用户同一优惠券领取的时间集合 ** **t7 : 113640 样本数

/Users/jimmy233/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [79]:
def get_day_gap_before(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(
            int(date_received[0:4]), int(date_received[4:6]),
            int(date_received[6:8])) - date(
                int(d[0:4]), int(d[4:6]), int(d[6:8]))).days
        if this_gap > 0:                   #只考虑以前领取的
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)


def get_day_gap_after(s):
    date_received, dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]), int(d[4:6]), int(d[6:8])) - date(
            int(date_received[0:4]), int(date_received[4:6]),
            int(date_received[6:8]))).days
        if this_gap > 0:                    #只考虑之后领取的
            gaps.append(this_gap)
    if len(gaps) == 0:
        return -1
    else:
        return min(gaps)


t7 = dataset3[['user_id', 'coupon_id', 'date_received']]
t7 = pd.merge(t7, t6, on=['user_id', 'coupon_id'], how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[[
    'user_id', 'coupon_id', 'date_received', 'day_gap_before', 'day_gap_after'
]]
# t7前后一次领取优惠券间隔时间

In [81]:
other_feature3 = pd.merge(t1, t, on='user_id')
other_feature3 = pd.merge(other_feature3, t3, on=['user_id', 'coupon_id'])
other_feature3 = pd.merge(other_feature3, t4, on=['user_id', 'date_received'])
other_feature3 = pd.merge(
    other_feature3, t5, on=['user_id', 'coupon_id', 'date_received'])
other_feature3 = pd.merge(
    other_feature3, t7, on=['user_id', 'coupon_id', 'date_received'])
other_feature3.to_csv('data/other_feature3.csv', index=None)

## dataset2 other feature

In [80]:
def get_feature(dataset):
    #for dataset
    #dataset['date_received'] = dataset['date_received'].apply(lambda x : str(int(float(x))))
    t = dataset[['user_id']]
    t['this_month_user_receive_all_coupon_count'] = 1
    t = t.groupby('user_id').agg('sum').reset_index()

    t1 = dataset[['user_id','coupon_id']]
    t1['this_month_user_receive_same_coupon_count'] = 1
    t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()

    t2 = dataset[['user_id', 'coupon_id', 'date_received']]
    t2.date_received = t2.date_received.astype('str')
    t2 = t2.groupby(
        ['user_id',
         'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t2['receive_number'] = t2.date_received.apply(lambda s: len(s.split(':')))
    t2 = t2[t2.receive_number > 1]        #保证至少有两条领取记录,代表一定使用过了上一张优惠券
    t2['max_date_received'] = t2.date_received.apply(
        lambda s: max([int(d) for d in s.split(':')]))
    t2['min_date_received'] = t2.date_received.apply(
        lambda s: min([int(d) for d in s.split(':')]))
    t2 = t2[['user_id', 'coupon_id', 'max_date_received', 'min_date_received']]

    t3 = dataset[['user_id', 'coupon_id', 'date_received']]
    t3 = pd.merge(t3, t2, on=['user_id', 'coupon_id'], how='left')
    t3.date_received = t3.date_received.apply(lambda x : float(x))
    t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
    t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received


    def is_firstlastone(x):
        if x == 0:
            return 1
        elif x > 0:
            return 0
        else:
            return -1  #those only receive once


    t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(
        is_firstlastone)
    t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(
        is_firstlastone)
    t3 = t3[[
        'user_id', 'coupon_id', 'date_received',
        'this_month_user_receive_same_coupon_lastone',
        'this_month_user_receive_same_coupon_firstone'
    ]]

    t3['date_received'] = t3['date_received'].apply(lambda x : str(int(x)))   #将t3的时间格式转换为str
    
    
    t4 = dataset[['user_id','date_received']]
    t4['this_day_user_receive_all_coupon_count'] = 1
    t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()

    t5 = dataset[['user_id','coupon_id','date_received']]
    t5['this_day_user_receive_same_coupon_count'] = 1
    t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()

    t6 = dataset[['user_id', 'coupon_id', 'date_received']]
    t6.date_received = t6.date_received.astype('str')
    t6 = t6.groupby(
        ['user_id',
         'coupon_id'])['date_received'].agg(lambda x: ':'.join(x)).reset_index()
    t6.rename(columns={'date_received': 'dates'}, inplace=True)

    def get_day_gap_before(s):
        date_received, dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(
                int(date_received[0:4]), int(date_received[4:6]),
                int(date_received[6:8])) - date(
                    int(d[0:4]), int(d[4:6]), int(d[6:8]))).days
            if this_gap > 0:                   #只考虑以前领取的
                gaps.append(this_gap)
        if len(gaps) == 0:
            return -1
        else:
            return min(gaps)


    def get_day_gap_after(s):
        date_received, dates = s.split('-')
        dates = dates.split(':')
        gaps = []
        for d in dates:
            this_gap = (date(int(d[0:4]), int(d[4:6]), int(d[6:8])) - date(
                int(date_received[0:4]), int(date_received[4:6]),
                int(date_received[6:8]))).days
            if this_gap > 0:                    #只考虑之后领取的
                gaps.append(this_gap)
        if len(gaps) == 0:
            return -1
        else:
            return min(gaps)


    t7 = dataset[['user_id', 'coupon_id', 'date_received']]
    t7 = pd.merge(t7, t6, on=['user_id', 'coupon_id'], how='left')
    t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
    t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
    t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
    t7 = t7[[
        'user_id', 'coupon_id', 'date_received', 'day_gap_before', 'day_gap_after'
    ]]
    
    other_feature = pd.merge(t1,t,on='user_id')
    other_feature = pd.merge(other_feature,t3,on=['user_id','coupon_id'])
    other_feature = pd.merge(other_feature,t4,on=['user_id','date_received'])
    other_feature = pd.merge(other_feature,t5,on=['user_id','coupon_id','date_received'])
    other_feature = pd.merge(other_feature,t7,on=['user_id','coupon_id','date_received'])
#     other_feature.to_csv('data/other_feature2.csv',index=None)
#     print(other_feature2.shape)
    return other_feature

In [82]:
other_feature2 = get_feature(dataset2)
# print('shape :',other_feature2.shape)
# print('columns : ',other_feature2.columns)
other_feature2.to_csv('data/other_feature2.csv',index=None)

<ipython-input-80-67c2dc43930f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['this_month_user_receive_all_coupon_count'] = 1
<ipython-input-80-67c2dc43930f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['this_month_user_receive_same_coupon_count'] = 1
/Users/jimmy233/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

## dataset1 other feature

In [83]:
other_feature1 = get_feature(dataset1)


other_feature1.to_csv('data/other_feature1.csv',index=None)

<ipython-input-80-67c2dc43930f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['this_month_user_receive_all_coupon_count'] = 1
<ipython-input-80-67c2dc43930f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['this_month_user_receive_same_coupon_count'] = 1
<ipython-input-80-67c2dc43930f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

# merchant related feature

## dataset3 merchant feature

In [84]:
#for dataset3
merchant3 = feature3[[
    'merchant_id', 'coupon_id', 'distance', 'date_received', 'date'
]]

t = merchant3[['merchant_id']]
t.drop_duplicates(inplace=True)

#每商家总消费记录数目
t1 = merchant3[merchant3.date != 'null'][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
#商家优惠券使用记录
t2 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][[
    'merchant_id'
]]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
#每家店发出多少优惠券
t3 = merchant3[merchant3.coupon_id != 'null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
#商家被核销优惠券中的平均/最小/最大/中位数用户-商家距离
t4 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][[
    'merchant_id', 'distance'
]]
t4.replace('null', -1, inplace=True)
t4.distance = t4.distance.astype('int')
t4.replace(-1, np.nan, inplace=True)
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance': 'merchant_min_distance'}, inplace=True)

t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance': 'merchant_max_distance'}, inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance': 'merchant_mean_distance'}, inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance': 'merchant_median_distance'}, inplace=True)

merchant3_feature = pd.merge(t, t1, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t2, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t3, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t5, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t6, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t7, on='merchant_id', how='left')
merchant3_feature = pd.merge(
    merchant3_feature, t8, on='merchant_id', how='left')
merchant3_feature.sales_use_coupon = merchant3_feature.sales_use_coupon.replace(
    np.nan, 0)  #fillna with 0
#商家优惠券核销率
merchant3_feature[
    'merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon.astype(
        'float') / merchant3_feature.total_coupon
#商家销售中使用优惠券概率
merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon.astype(
    'float') / merchant3_feature.total_sales

merchant3_feature.total_coupon = merchant3_feature.total_coupon.replace(
    np.nan, 0)  #fillna with 0

<ipython-input-84-234cbc228e30>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t.drop_duplicates(inplace=True)


In [85]:
merchant3_feature.to_csv('data/merchant3_feature.csv', index=None)

## dataset2 merchant feature

In [86]:
#for dataset2
merchant2 = feature2[['merchant_id','coupon_id','distance','date_received','date']]

t = merchant2[['merchant_id']]
t.drop_duplicates(inplace=True)

t1 = merchant2[merchant2.date!='null'][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()

t2 = merchant2[(merchant2.date!='null')&(merchant2.coupon_id!='null')][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()

t3 = merchant2[merchant2.coupon_id!='null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()

t4 = merchant2[(merchant2.date!='null')&(merchant2.coupon_id!='null')][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.distance = t4.distance.astype('int')
t4.replace(-1,np.nan,inplace=True)
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace=True)

t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace=True)

merchant2_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t2,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t3,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t5,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t6,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t7,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t8,on='merchant_id',how='left')
merchant2_feature.sales_use_coupon = merchant2_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant2_feature['merchant_coupon_transfer_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_coupon
merchant2_feature['coupon_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_sales
merchant2_feature.total_coupon = merchant2_feature.total_coupon.replace(np.nan,0) #fillna with 0


<ipython-input-86-cb32700f20e8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t.drop_duplicates(inplace=True)


In [87]:
merchant2_feature.to_csv('data/merchant2_feature.csv',index=None)

## dataset1 merchant feature

In [88]:
#for dataset1
merchant1 = feature1[['merchant_id','coupon_id','distance','date_received','date']]

t = merchant1[['merchant_id']]
t.drop_duplicates(inplace=True)

t1 = merchant1[merchant1.date!='null'][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()

t2 = merchant1[(merchant1.date!='null')&(merchant1.coupon_id!='null')][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()

t3 = merchant1[merchant1.coupon_id!='null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()

t4 = merchant1[(merchant1.date!='null')&(merchant1.coupon_id!='null')][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.distance = t4.distance.astype('int')
t4.replace(-1,np.nan,inplace=True)
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace=True)

t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace=True)


merchant1_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t2,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t3,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t5,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t6,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t7,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t8,on='merchant_id',how='left')
merchant1_feature.sales_use_coupon = merchant1_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant1_feature['merchant_coupon_transfer_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_coupon
merchant1_feature['coupon_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_sales
merchant1_feature.total_coupon = merchant1_feature.total_coupon.replace(np.nan,0) #fillna with 0


<ipython-input-88-29af4d10b836>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t.drop_duplicates(inplace=True)


In [89]:
merchant1_feature.to_csv('data/merchant1_feature.csv',index=None)